# Day 3: The Chain Rule — Composing Gradients

**Learning Objective**: Implement the chain rule to propagate gradients backward through a computation graph automatically.

Today we'll add `_backward` functions to each operation and implement `backward()` to compute all gradients in one call!

In [ ]:
import math
from graphviz import Digraph

## The Chain Rule

For a composition $y = f(g(x))$:

$$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$$

**Key insight**: Each operation knows its *local gradient*. We multiply by the *upstream gradient* to get the *downstream gradient*.

**Important**: Always use `+=` for gradient accumulation (a variable used multiple times gets gradients from all paths).

## Value Class with Automatic Backward Pass

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None  # Will be set by operations
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            # Local gradient of addition is 1 for both inputs
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            # d(a*b)/da = b, d(a*b)/db = a
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            # d(tanh(x))/dx = 1 - tanh(x)^2
            self.grad += (1 - t**2) * out.grad
        
        out._backward = _backward
        return out

    def backward(self):
        """Compute gradients for all nodes using topological sort."""
        # Step 1: Build topological order
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        
        # Step 2: Set output gradient to 1.0
        self.grad = 1.0
        
        # Step 3: Backward pass in reverse topological order
        for node in reversed(topo):
            node._backward()

## Visualization Helper

In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name=uid, label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=uid + n._op, label=n._op)
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot

## Test 1: Simple Chain Rule

Expression: `L = (a * b + c) * f`

Let's verify that `backward()` computes the same gradients as our manual calculation from Day 2.

In [ ]:
# Build the expression
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a * b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'

# Automatic backward pass!
L.backward()

# Verify gradients
print("Gradients computed by backward():")
print(f"  L.grad = {L.grad:.4f} (should be 1.0)")
print(f"  f.grad = {f.grad:.4f} (should be 4.0)")
print(f"  d.grad = {d.grad:.4f} (should be -2.0)")
print(f"  c.grad = {c.grad:.4f} (should be -2.0)")
print(f"  e.grad = {e.grad:.4f} (should be -2.0)")
print(f"  a.grad = {a.grad:.4f} (should be 6.0)")
print(f"  b.grad = {b.grad:.4f} (should be -4.0)")

# Visualize
draw_dot(L)

## Test 2: Gradient Accumulation

When a variable is used multiple times, gradients must **accumulate** (that's why we use `+=`).

Expression: `b = a * a` (same as `a²`)

Expected: `∂b/∂a = 2a = 2 * 3 = 6`

In [ ]:
a = Value(3.0, label='a')
b = a * a; b.label = 'b'

b.backward()

print(f"a.grad = {a.grad:.4f} (should be 6.0 = 2 * 3)")
assert abs(a.grad - 6.0) < 1e-5, "Gradient accumulation failed!"
print("✅ Gradient accumulation works!")

## Test 3: Numerical Gradient Verification

We verify our analytical gradients by comparing with numerical approximation.

In [ ]:
def verify_gradient(name, analytical, numerical):
    match = abs(analytical - numerical) < 1e-4
    status = "✅" if match else "❌"
    print(f"{status} {name}: analytical={analytical:.4f}, numerical={numerical:.4f}")
    return match

def compute_L(a_val, b_val, c_val, f_val):
    """Helper to compute L = (a*b + c) * f"""
    return (a_val * b_val + c_val) * f_val

h = 1e-5

# Get analytical gradients
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a * b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L.backward()

# Compute numerical gradients
a_val, b_val, c_val, f_val = 2.0, -3.0, 10.0, -2.0

num_a = (compute_L(a_val + h, b_val, c_val, f_val) - compute_L(a_val - h, b_val, c_val, f_val)) / (2 * h)
num_b = (compute_L(a_val, b_val + h, c_val, f_val) - compute_L(a_val, b_val - h, c_val, f_val)) / (2 * h)
num_c = (compute_L(a_val, b_val, c_val + h, f_val) - compute_L(a_val, b_val, c_val - h, f_val)) / (2 * h)
num_f = (compute_L(a_val, b_val, c_val, f_val + h) - compute_L(a_val, b_val, c_val, f_val - h)) / (2 * h)

print("Gradient Verification:")
all_pass = all([
    verify_gradient("a.grad", a.grad, num_a),
    verify_gradient("b.grad", b.grad, num_b),
    verify_gradient("c.grad", c.grad, num_c),
    verify_gradient("f.grad", f.grad, num_f),
])

if all_pass:
    print("\n🎉 All gradients verified!")

## Test 4: Neuron with Automatic Backprop

Let's verify backward() works with tanh activation: `o = tanh(x1*w1 + x2*w2 + b)`

In [ ]:
# Inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# Weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# Bias (chosen so tanh output is ~0.7071)
b = Value(6.8813735870195432, label='b')

# Forward pass
x1w1 = x1 * w1; x1w1.label = 'x1*w1'
x2w2 = x2 * w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1+x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

# Backward pass - automatic!
o.backward()

print("Neuron gradients (automatic):")
print(f"  x1.grad = {x1.grad:.4f}")
print(f"  x2.grad = {x2.grad:.4f}")
print(f"  w1.grad = {w1.grad:.4f}")
print(f"  w2.grad = {w2.grad:.4f}")
print(f"  b.grad  = {b.grad:.4f}")

# Visualize
draw_dot(o)

## Summary

Today we implemented:

1. **`_backward` closures** for each operation (add, mul, tanh)
2. **Topological sort** to process nodes in correct order
3. **`backward()` method** that computes all gradients automatically
4. **Gradient accumulation** using `+=` for variables used multiple times

This is the foundation of automatic differentiation used in PyTorch, TensorFlow, and all modern deep learning frameworks!

---

*Next up: Day 4 - More Operations (subtraction, division, power, exp)*